In [54]:
cd /home/icb/alessandro.palma/environment/flatvi_baselines/phate_fim

/ictstr01/home/icb/alessandro.palma/environment/flatvi_baselines/phate_fim


In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import scanpy as sc
import numpy as np
import phate 
import scipy as sp
from scipy.spatial.distance import pdist, squareform
from data import train_valid_loader_from_pc
from model import AEDist
import torch
from transformations import LogTransform, NonTransform, StandardScaler, \
    MinMaxScaler, PowerTransformer, KernelTransform

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from torch.utils.data import Dataset

import sys 
sys.path.insert(0, "/home/icb/alessandro.palma/environment/flatvi_baselines/phate_fim/src")
from src.models.lit_encoder import LitAutoencoder

In [57]:
class torch_dataset(Dataset):
    def __init__(self, X, Y) -> None:
        self.X = X
        self.Y = Y
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        target = self.Y[index, :]
        sample = self.X[index, :]
        return sample, target

In [58]:
adata = sc.read_h5ad("/home/icb/alessandro.palma/environment/scCFM/project_dir/data/pancreas/processed/pancreas.h5ad")

In [29]:
adata.X.shape

(20519, 2000)

In [30]:
X_expression = np.array(adata.X.copy().todense())

### Run PHATE

In [31]:
phate_op = phate.PHATE()
phate_coords = phate_op.fit_transform(X_expression)

Calculating PHATE...
  Running PHATE on 20519 observations and 2000 variables.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 3.35 seconds.
    Calculating KNN search...
    Calculated KNN search in 31.48 seconds.
    Calculating affinities...
    Calculated affinities in 3.54 seconds.
  Calculated graph and diffusion operator in 38.41 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 2.44 seconds.
    Calculating KMeans...
    Calculated KMeans in 3.36 seconds.
  Calculated landmark operator in 6.96 seconds.
  Calculating optimal t...
    Automatically selected t = 19
  Calculated optimal t in 1.67 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.62 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 5.72 seconds.
Calculated PHATE in 53.39 seconds.


In [32]:
phate_coords = sp.stats.zscore(phate_coords) 

In [33]:
X_expression = torch.from_numpy(X_expression)
phate_coords = torch.from_numpy(phate_coords)
train_dataset = torch_dataset(X_expression, phate_coords)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=256, shuffle=True
)

In [70]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = Args(
    run_name=None,
    dataset="tree",
    n_obs=1600,
    n_dim=2000,
    batch_size=150,
    lr=0.0001,
    knn=5,
    max_epochs=150,
    wandb=False,
    activation="ReLU",
    kernel_type="phate",
    loss="loss_dist",
    encoder_layer=[256, 100, 10],
    scale=0.0005,
    inference=False,
    inference_obs=1600
)

dict_args = vars(args)

In [71]:
logger = WandbLogger(project="fim_phate", name=args.run_name)

In [72]:
trainer = Trainer.from_argparse_args(
    args, accelerator="gpu", devices=1, logger=logger
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [73]:
emb_dim = args.encoder_layer[-1]
model = LitAutoencoder(input_dim=args.n_dim, emb_dim=emb_dim, **dict_args)
trainer.fit(model, train_dataloaders=train_loader)
model_name = os.getcwd() + args.dataset + '.pt'
torch.save(model.state_dict(), model_name)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 543 K 
1 | decoder | Sequential | 5.8 K 
---------------------------------------
548 K     Trainable params
0         Non-trainable params
548 K     Total params
2.195     Total estimated model params size (MB)


[Linear(in_features=2000, out_features=256, bias=True), ReLU(), Linear(in_features=256, out_features=100, bias=True), ReLU(), Linear(in_features=100, out_features=50, bias=True), Softmax(dim=1)]
[Linear(in_features=50, out_features=50, bias=True), ReLU(), Linear(in_features=50, out_features=50, bias=True), ReLU(), Linear(in_features=50, out_features=10, bias=True), ReLU(), Linear(in_features=10, out_features=10, bias=True), ReLU(), Linear(in_features=10, out_features=10, bias=True), ReLU()]


Training: 0it [00:00, ?it/s]

/home/icb/alessandro.palma/miniconda3/envs/flatvi_baselines_env/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


NameError: name 'os' is not defined